In [ ]:
import hoomd
import hoomd.md
import sys
import gsd
import gsd.hoomd
import hoomd.md.bond
import mbuild as mb
hoomd.context.initialize("/home/cphollarbush/mbuild.gsd")

#imports all the necessary modules, so we can actually use them
#hoomd is the overarching engine; hoomd.md is the molecular dynamics portion of hoomd; sys allows us to use system commands; gsd and gsd.hoomd are needed to work with gsd files; hoomd.md.bond allows us to work with bond data
#hoomd.context.initialize gets the system up and running for use

In [ ]:
#box = mb.load("/home/cphollarbush/Avo_atoms/airhcn1.pdb")
#box.save("/home/cphollarbush/Avo_atoms/airhcn1.gsd")
#print("it worked!")

#'translates' the pdb file into a gsd so hoomd can read it

In [ ]:
system = hoomd.init.read_gsd("/home/cphollarbush/mbuild.gsd")

#creates a system which is comprised of whatever is in the gsd
#the file is the one with ATOM instead of HETAM

In [ ]:
nl = hoomd.md.nlist.cell();

#tells hoomd to calculate forces only in a certain proximity; saves computer power w/o reducing accuracy

In [ ]:
lj = hoomd.md.pair.lj(r_cut=1.5, nlist=nl);

#tells hoomd we're using LJ equations
#sets the nlist radius at 2.5 angstroms

In [ ]:
lj.pair_coeff.set('N','N', epsilon=1.0, sigma=1.0);
lj.pair_coeff.set('N','0', epsilon=1.0, sigma=1.0);
lj.pair_coeff.set('N','AR', epsilon=1.0, sigma=1.0);
lj.pair_coeff.set('N','H', epsilon=1.0, sigma=1.0);
lj.pair_coeff.set('N','C', epsilon=1.0, sigma=1.0);

lj.pair_coeff.set('O','O', epsilon=1.0, sigma=1.0);
lj.pair_coeff.set('O','AR', epsilon=1.0, sigma=1.0);
lj.pair_coeff.set('O','C', epsilon=1.0, sigma=1.0);
lj.pair_coeff.set('O','H', epsilon=1.0, sigma=1.0);

lj.pair_coeff.set('AR','AR', epsilon=1.0, sigma=1.0);
lj.pair_coeff.set('AR','H', epsilon=1.0, sigma=1.0);
lj.pair_coeff.set('AR','N', epsilon=1.0, sigma=1.0);
lj.pair_coeff.set('AR','C', epsilon=1.0, sigma=1.0);

lj.pair_coeff.set('H','C', epsilon=1.0, sigma=1.0);
lj.pair_coeff.set('H','H', epsilon=1.0, sigma=1.0);
lj.pair_coeff.set('C','N', epsilon=1.0, sigma=1.0);
lj.pair_coeff.set('C','C', epsilon=1.0, sigma=1.0);
lj.pair_coeff.set('N','O', epsilon=1.0, sigma=1.0);

#data for the LJ forces in the sim
#will replace demo epsilon and sigma values with actual values once simulation runs

In [ ]:
#'N' = 1
#'O' = 2
#"H" = 3
#"C" = 4

#attempting to name each type of atom in hopes that it will allow us to create bonds

In [ ]:
all = hoomd.group.all();

#makes it easier for us to group all particles together

In [ ]:
#system.bonds.add("NN", 1, 1)
#system.bonds.add("OO", 2, 2)
#system.bonds.add("HC", 3, 4)
#system.bonds.add("CN", 4, 1)

#attempting to add bonds
#name in red and particlescomprised of in green. see above for relationship between number and particle type

In [ ]:
hoomd.md.integrate.mode_standard(dt=.1) 

integrator = hoomd.md.integrate.npt(group=all, kT=2.49433795843, tau=138.08, tauP=1.0, P=2.0) 

integrator.randomize_velocities(seed=42)

#setting up integrator
#integrator takes in data: kT, tau, tauP
#randomize velocities is what uses the integrator + its data then assigns velocities

#will replace demo values with actual values once simulation runs

In [ ]:
fene = hoomd.md.bond.fene();
fene.bond_coeff.set('N-N', k=30.0, r0=1.5, sigma=1.0, epsilon= 2.0);
fene.bond_coeff.set('O-O', k=30.0, r0=1.5, sigma=1.0, epsilon= 2.0);
fene.bond_coeff.set('C-H', k=30.0, r0=1.5, sigma=1.0, epsilon= 2.0);
fene.bond_coeff.set('C-N', k=30.0, r0=1.5, sigma=1.0, epsilon= 2.0);

#attempt to specify bond forces
#will replace demo values with actual values once simulation runs

In [ ]:
hoomd.run(1e4);

#what is actually used to run the simulation

In [ ]:
snapshot = system.take_snapshot(particles=True, bonds=True, pairs=False, integrators=True, all=True, dtype='float')

#snapshot takes a "picture" of one time step of the simulation which can be used for analysis of all the above data types in parentheses

In [ ]:
print(snapshot.particles.velocity)
#our method of checking to see if the integrator worked and the particles have velocities

In [ ]:
all = hoomd.group.all();
hoomd.dump.gsd(filename="/home/cphollarbush/Avo_atoms/trajectory.gsd", period=10, group = all, phase=0)
#save(filename="trajectory.gsd"/home/cphollarbush/Avo_atoms/trajectory.gsd)

#takes the simulation and creates a file with all the data in it
#saves that file to the location we designate